In [10]:
#CLSP-Rmin with back-ordering example
from gurobipy import *
import xlsxwriter
import time
import math
import random
from function_call import compute_values
# from function_call_clsp import compute_values as cpCLSP 
from data import variables
import json
import numpy as np


#start = time.time()

n = Model("CLSP-RminBO")
n.Params.LogToConsole = 0

#Parameters
d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe = variables()

#realized plan - create empty containers
Items = len(Pro)
Periods = len(Pe)
lotsizes = np.zeros((Items, Periods))
lotsizesR = np.zeros((Items, Periods))

inventory = np.zeros((Items, Periods))
inventoryR = np.zeros((Items, Periods))

backorders = np.zeros((Items, Periods))

setups = np.zeros((Items, Periods))
setupsR = np.zeros((Items, Periods))

start = 0

# generate random ratio for defective items
random.seed(123)
o1 = round(random.uniform(0.01,0.02), 3)# realistic
o2 = round(random.uniform(0.02,0.03), 3) # pessimistic 
o3 = round(random.uniform(0.001,0.01), 3) # optimistic 
O=[o1 for t in Pe] # realistic
O1=[o2 for t in Pe ]  # pessimistic
O2=[o3 for t in Pe] # optimistic 

realizationO= [round(random.uniform(0.01,0.02),3) for t in Pe]
realizationO1= [round(random.uniform(0.02,0.03),3) for t in Pe]
realizationO2= [round(random.uniform(0.001,0.01),3) for t in Pe]

lista=[{'O':O, 'O1':O1,'O2':O2,'realizationO':realizationO,
        'realizationO1':realizationO1,'realizationO2':realizationO2}]
with open('data.json', 'w') as fa:
    json.dump(lista,fa)
json_str = json.dumps(lista)
#print(json_str)
with open('data.json', 'r') as fa:
    thisfile=json.load(fa)
O=thisfile[0]["O"]
realizationO=thisfile[0]["realizationO"]


#################################### Run optymalization ############################# 


#Pe=range(T)    
#CostBO=True
#Rp=10
#cpCLSP(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,CostBO)


T=6    
Pe=range(T)    
Rp=2


A1 = 0
B1 = 0
C1 = 0
D1 = 0
E1 = 0 
F1 = 0
G1 = 0
H1 = 0
I1 = 0
J1 = 0
K1 = 0

oH = 0
ohR = 0
cC = 0
oF = 0
oxprod = 0
fcostr = 0
oxrework = 0 
oCU = 0
ZZ = 0
cbcost = 0
cbcost2 = 0




for i in range(5):
    A1 = A1 + oH
    B1 = B1 + ohR
    C1 = C1 + cC
    D1 = D1 + oF
    E1 = E1 + oxprod
    F1 = F1 + fcostr
    G1 = G1 + oxrework
    H1 = H1 +  oCU
    I1 = I1 + ZZ
    J1 = J1 + cbcost
    K1 = K1 + cbcost2
    if i ==0:
        print('step',i+1)
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2,
        yS, yR, s, sR, BO, x, xR] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)
        
        for item in Pro:
            for t in range(start, start+Rp):
                lotsizes[item, t] = yS[item, t-start]
                lotsizesR[item, t] = yR[item,t-start].x
                inventory[item, t] = s[item, t-start]
                inventoryR[item, t] = sR[item, t-start]
                backorders[item, t] = BO[item, t-start]
                setups[item, t] = x[item, t-start].x
                setupsR[item, t] = xR[item, t-start].x
        start += Rp    
        
        
    
    elif i==3: #and T >len(b) and Rp<len(b):
        print('step',i+1)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)
        #print(BO0)
        #print(s0)
        #print(sR0)
        #print(b)
        #print(d)
        Pe=range(len(b))
        T=len(b)
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2,
        yS, yR, s, sR, BO, x, xR] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)
        
        for item in Pro:
            for t in range(start, start+Rp):
                lotsizes[item, t] = yS[item, t-start]
                lotsizesR[item, t] = yR[item,t-start].x
                inventory[item, t] = s[item, t-start]
                inventoryR[item, t] = sR[item, t-start]
                backorders[item, t] = BO[item, t-start]
                setups[item, t] = x[item, t-start].x
                setupsR[item, t] = xR[item, t-start].x
        start += Rp   
        
    elif i==4: #T>len(b) and Rp>len(b)
        print('step',i+1)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)
        Pe=range(len(b))
        T=len(b)
        Rp=len(b)
        #print(BO0)
        #print(s0)
        #print(sR0)
        #print(b)
        #print(d)
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2,
        yS, yR, s, sR, BO, x, xR] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)
        
        for item in Pro:
            for t in range(start, min(start+Rp, Periods)):
                lotsizes[item, t] = yS[item, t-start]
                lotsizesR[item, t] = yR[item,t-start].x
                inventory[item, t] = s[item, t-start]
                inventoryR[item, t] = sR[item, t-start]
                backorders[item, t] = BO[item, t-start]
                setups[item, t] = x[item, t-start].x
                setupsR[item, t] = xR[item, t-start].x
        start += Rp   
    
    
    
    elif i<3:
        print('step',i+1, 'start', start)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)
        #print(i)
        #print(len(b))
        #print(b)
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2, 
         y, yR, s, sR, BO, x, xR] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)
        for item in Pro:
            for t in range(start, start+Rp):
                lotsizes[item, t] = y[item, t-start]
                lotsizesR[item, t] = yR[item,t-start].x
                inventory[item, t] = s[item, t-start]
                inventoryR[item, t] = sR[item, t-start]
                backorders[item, t] = BO[item, t-start]
                setups[item, t] = x[item, t-start].x
                setupsR[item, t] = xR[item, t-start].x
        start += Rp   
        
        
    else:
        print('end')
        

hcost = inventory.sum(axis=1).dot(h)
hcostR = inventoryR.sum(axis=1).dot(hR)
scost = setups.sum(axis=1).dot(f)
scostR = setupsR.sum(axis=1).dot(f)
bcost = backorders.sum(axis=1).dot(np.array(h)*m)

print("REALIZED PLAN")
print('Total cost:', hcost+hcostR+scost+scostR+bcost)
print('Holding cost (good items):', hcost, 'Holding cost (defect items):', hcostR)
print('Setup cost (good items):', scost, 'Setup cost (defect items):', scostR)
print('Backorder cost:', bcost)

print('Production plan:\n', lotsizes, '\nProduction plan to rework items:\n', lotsizesR)
print('Inventory (good items):\n', inventory, '\nInventory (defect items):\n', inventoryR)
print('Realized backorders:\n', backorders)

workbook = xlsxwriter.Workbook('out.xlsx')
# The workbook object is then used to add new
# worksheet via the add_worksheet() method.
worksheet = workbook.add_worksheet()
 
# Use the worksheet object to write
# data via the write() method.
worksheet.write('A1', "h Holding cost")
worksheet.write('B1', "hR Holding cost for rework")
worksheet.write('C1', "Setup cost combined")
worksheet.write('D1', "f Setup cost for production")
worksheet.write('E1', "x Nr of set-ups for production")
worksheet.write('F1', "f Setup cost for rework")
worksheet.write('G1', "xR Setup cost for rework")
worksheet.write('H1', "Capacity Utilization")
worksheet.write('I1', "Z objective function value")
worksheet.write('J1', "Back order costs cb")
worksheet.write('K1', "Back order realization costs cb")
worksheet.write('A2',A1)
worksheet.write('B2',B1)
worksheet.write('C2',C1)
worksheet.write('D2',D1)
worksheet.write('E2',E1)
worksheet.write('F2',F1)
worksheet.write('G2',G1)
worksheet.write('H2',H1/5)
worksheet.write('I2',I1)
worksheet.write('J2',J1)
worksheet.write('K2',K1)
# Finally, close the Excel file
# via the close() method.
workbook.close()        
        


step 1
Lots rework:
 [[-0. -0.  0.  0. -0. 18.]
 [-0. -0. -0. -0. -0. 23.]
 [-0. -0. 17. -0.  0. 18.]
 [-0.  0.  0. -0. -0. 35.]]
Setups rework:
 [[ 0. -0.  0. -0. -0.  1.]
 [ 0.  0. -0. -0. -0.  1.]
 [-0. -0.  1. -0. -0.  1.]
 [ 0. -0. -0.  0. -0.  1.]]
O 0.011
O 0.019
O 0.01
O 0.015
O 0.013
O 0.019
O 0.011
O 0.019
O 0.01
O 0.015
O 0.013
O 0.019
O 0.011
O 0.019
O 0.01
O 0.015
O 0.013
O 0.019
O 0.011
O 0.019
O 0.01
O 0.015
O 0.013
O 0.019
step 2 start 2
Lots rework:
 [[ 0.  0.  0.  0. 28.  0.]
 [ 0.  0.  0.  0.  0. 37.]
 [21.  0.  0.  0.  0. 34.]
 [ 0.  0.  0.  0.  0. 54.]]
Setups rework:
 [[0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]]
O 0.011
O 0.019
O 0.01
O 0.015
O 0.013
O 0.019
O 0.011
O 0.019
O 0.01
O 0.015
O 0.013
O 0.019
O 0.011
O 0.019
O 0.01
O 0.015
O 0.013
O 0.019
O 0.011
O 0.019
O 0.01
O 0.015
O 0.013
O 0.019
step 3 start 4
Lots rework:
 [[ 0.  0. 38.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 49.]
 [ 0.  0.  0.  0.  0. 53.]
 [ 0.  0.  0.  0. 66